# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902000


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:01<00:23,  1.21it/s]

Rendering models:  10%|▉         | 3/31 [00:06<01:00,  2.17s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:46,  1.71s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:32,  1.27s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:14<00:00,  3.30it/s]

Rendering models:  94%|█████████▎| 29/31 [00:14<00:00,  3.28it/s]

Rendering models:  97%|█████████▋| 30/31 [00:15<00:00,  2.55it/s]

Rendering models: 100%|██████████| 31/31 [00:15<00:00,  2.19it/s]

equidad1                              0.000986
CTidwell3                             0.000304
klmasters                             0.000380
YugnatD                               0.001771
not-logged-in-d254cc3304773762c2aa    0.396102
not-logged-in-dacb6c2f9eb6080c5e9f    0.000438
not-logged-in-745edeb38ef217ac29f9    0.001555
not-logged-in-61c60101aa7fea3372e3    0.000302
not-logged-in-b2ab6eb00573e0111cbd    0.000583
dianapenguinp                         0.000575
TheoBe                                0.000296
not-logged-in-7cc17a8a1e6ff5a23d80    0.000978
mariafarhat                           0.002212
not-logged-in-d05770f05ee9ee0b9740    0.001317
not-logged-in-adaa8793e217e17a4a47    0.000362
UnbiasedBrigade                       0.001546
not-logged-in-d04ed8c60cd5453cb408    0.000416
BraxtonKinney                         0.000317
acapirala                             0.000480
not-logged-in-3569dcff4dbd0ee90f50    0.001020
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000688
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())